# ロケット打ち上げはどこまで見えるのか？国土地理院のオープンデータから調べてみた

発射されたロケットはどこから見えるか判定するコードです。

詳しくはこちら、https://sorabatake.jp/xxxxxx/  をご覧下さい。


## 実行の仕方
JupyterLabは対話型のコード実行環境です。各セルを「Shift + Enter」で順に実行して下さい。  

## 計算概要
1. 2地点間の直線上の緯度経度を取得。
2. 2地点間の標高をAPIで取得。
3. 2地点間の視線を、ロケットの高さと観測地点からの距離で割り出す。
    1. 底辺：2地点間の距離出す
    2. 高さ：ロケットの標高
    3. 角度：tanθ
4. 2地点間の標高と視線を比較。標高の方が大きければ見通し不可。

## ロケットの発射上の緯度経度と打ち上げ時の高さを設定

In [1]:
lon1,lat1 = 139.81069, 35.71005 #ロケット発射場
height = 634 # ロケット打ち上げ時の高さを指定[m]
los_radius = 20 #見通しエリアの半径[km]
concentric_circle = 10 # 見通しエリアの同心円の数
accuracy = 100 # 見通し計算の精度

#発射場の緯度経度は発表資料から予測
#https://news.mynavi.jp/article/kushimoto-2/

## 2地点間の標高を取得する関数作成

In [2]:
# 2地点間の座標を取得
from pyproj import Geod
def get_midpoints(lon1,lat1,lon2,lat2,npts=10):
    """
    2地点間の緯度経度を取得
    
    lon1,lat1 : ロケットの座標
    lon2, lat2：観測点の座標
    npts: 取得する2点間の座標の数
    """
    geod = Geod("+ellps=WGS84")
    return geod.npts(lon1, lat1, lon2, lat2, npts)

# 座標から標高を取
import requests, json
def get_kokudochiriin_elevation(lon,lat):
    """
    国土地理院の標高データを取得
    """
    url = "https://cyberjapandata2.gsi.go.jp/general/dem/scripts/getelevation.php?lon={}&lat={}&outtype=JSON".format(lon,lat)
    r = requests.get(url)
    data =json.loads(r.content)
    return data['elevation']

## 2地点間の視線を計算する関数作成

In [3]:
def get_distance(lon1,lat1,lon2,lat2):
    """
    2点の距離を取得
    """
    grs80 = Geod(ellps='GRS80')  # GRS80楕円体
    azimuth, bkw_azimuth, distance = grs80.inv(lon1, lat1, lon2, lat2)
    return distance

import math
def get_angle(height,distance):
    """
    底辺と高さから角度（°）を取得
    """
    tan = height/distance
    return math.degrees(math.atan(tan))

def get_height(distance,angle):
    """
    底辺と角度から高さを取得
    """
    return distance * math.tan(math.radians(angle))

def getHorizonDistance(h0, earth_radius):
    if h0 < 0:
        h0 = 0
    return math.sqrt( math.pow(h0, 2) + 2 * earth_radius * h0 );

def getTargetHiddenHeight(d2, earth_radius):
    if d2 < 0:
        return 0;
    return math.sqrt(math.pow(d2, 2) + math.pow(earth_radius, 2)) - earth_radius;

def get_shizumikomi(observer_height, distance):
    """
    observer_height : 観測点の標高(m)
    earth_radius : 対象点との距離(m)
    """
    earth_radius = 8494.666 #光の屈折を踏まえた地球の半径4/3倍（ここでは光の屈折率を電磁波と同じと見なす） （実際の半径は6371km）
    
    observer_height_km = observer_height * 0.001 #地球半径の単位と合わせるため km に変換
    distance_km = distance * 0.001
    
    oberver_to_horizon_km = getHorizonDistance(observer_height_km, earth_radius);
    shizumikomi_km = getTargetHiddenHeight(distance_km - oberver_to_horizon_km, earth_radius);
    
    return shizumikomi_km * 1000; # m に直す

## 標高と視線を比較する関数作成

In [4]:
from tqdm.notebook import tqdm

def customize_return(elevation):
    if "-----" == elevation:
        return 0
    else:
        return elevation
    
def is_los_true(lon1,lat1,lon2,lat2,height,npts):
    """
    2地点間の見通しを計算
    
    lon1,lat1 : ロケットの座標
    lon2, lat2：観測点の座標
    npts: 標高を取得する座標の数
    
    return: los_true:見通せる緯度経度、los_false:見通し不可な緯度経度
    """
    # 2地点間の座標を取得
    mid_points = get_midpoints(lon1,lat1,lon2,lat2,npts)
    
    # 観測点も追加
    mid_points.append((lon2,lat2))
    
    #観測点→対象点に近い点の順に並び替え
    mid_points.reverse()
    
    # 中間点の数を取得(観測点を含む)
    mid_points_count = len(mid_points)
            
    # 観測点から、対称点の距離を取得
    distance = get_distance(lon1,lat1,lon2,lat2)
        
    # 中間点間の距離を取得
    span = distance/mid_points_count
        
    # 観測点からの視線計算の角度を取得
    observer_elevation = customize_return(get_kokudochiriin_elevation(lon2,lat2))

    #底辺をそろえる
    height = height - observer_elevation
    
    #見た目の高さを取得
    height = height - get_shizumikomi(observer_elevation,distance)

    #角度を取得
    angle = get_angle(height,distance)
        
    for index in range(mid_points_count):
        
        #中間点を取得
        mid_point = mid_points[index]
        
        #中間点から対象点までの距離
        span_sum = span * ( index + 1 )
                
        #現在の中間点における、視線の高さを計算
        mid_los_height = get_height(span_sum,angle)
        mid_los_height = mid_los_height + observer_elevation
        
        #現在の中間点における、標高を取得
        mid_elevation = customize_return(get_kokudochiriin_elevation(mid_point[0],mid_point[1]))
        mid_elevation = mid_elevation - get_shizumikomi(observer_elevation,span_sum)
        
        # 視線と中間の標高の比較
        if mid_los_height < float(mid_elevation):
            result = False
            break;
        else:
            result = True
            
    return result

In [5]:
from functools import partial
import pyproj
from shapely.ops import transform
from shapely.geometry import Point

proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')

def geodesic_point_buffer(lon, lat, km):
    """
    中心点から半径 N km の円周の緯度経度を取得
    
    lon1,lat1 : 中心点の座標
    km: 半径 (kmで指定)
    return: 円周の緯度経度の座標
    """
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(km * 1000)  # distance in metres
    return transform(project, buf).exterior.coords[:]

def get_concentric_circle_points_from_circumference(circumference_points,lon1,lat1, npts):
    """
    円周の外側の点から、同心円状に内側の点を取得
    circumference_points: 円周の外側の点 （lon,lat）
    lon1,lat1 : 中心点の座標
    npts : 同心円の数
    return: 同心円状の点の緯度軽度 [[(lon,lat),(lon,lat)...],[(lon,lat),(lon,lat)...]]
    """
    
    concentric_circle_points = []
        
    for circumference_point in circumference_points:
        #円周の内側の点を取得
        inner_points = get_midpoints(lon1,lat1,circumference_point[0],circumference_point[1],npts)
        #円周の外側の点も追加
        inner_points.append((circumference_point[0],circumference_point[1]))
        # 円周の特定の角度の外側から中心までの点を、配列に追加
        concentric_circle_points.append(inner_points)
        
    return concentric_circle_points    
    

In [ ]:
# pip install folium するのを忘れずに!
import folium

def get_los_area(lon1,lat1,height,rad_km, con_circle, accuracy):
    
    #対象点を中心とした円周上の外側の点の座標を取得
    search_areas_outer_points = geodesic_point_buffer(lon1, lat1, rad_km)#半径をkmで指定
    
    #対象点を中心とした同心円状の点の座標を取得
    concentric_circle_points = get_concentric_circle_points_from_circumference(search_areas_outer_points,lon1,lat1,con_circle)#同心円の数を指定
    
    concentric_circle_los = []
    points_true = []
    points_false = []        

    for i in tqdm(range(len(concentric_circle_points)), desc='見通しエリアを計算', leave=False):
        
        concentric_circle_point = concentric_circle_points[i]
        
        progress_radian = i * 5.45
        
        # ある角度の座標一覧を外側から見通し計算していく
        for point in tqdm(concentric_circle_point, desc='{}°上の直線の見通しを計算'.format(progress_radian)):

            result = is_los_true(lon1,lat1,point[0],point[1],height,accuracy)#中間ポイントの細かさを指定
            
            if result:
                points_true.append(point)
            else:
                points_false.append(point)
                
        concentric_circle_los.append({'los_true':points_true,'los_false':points_false})
        
    return concentric_circle_los

def draw_los_maker(lon1,lat1,height,rad_km,con_circle,accuracy,los_map):
    """
    見通しマーカーを地図上に描画
    """
    
    areas = get_los_area(lon1,lat1,height,rad_km,con_circle,accuracy)
                
    # 同心円上の座標を処理
    for area in tqdm(areas, desc='見通しマップを描画'):        
        for key in area:
                        
            for points in area[key]:
                
                # 計算結果をマーカーのカラーに反映
                if 'los_true' == key:
                    color='blue';
                    fill_color='#3186cc'
                else:
                    color='gray'
                    fill_color='#f5f5f5'
                    
                #観測点のマーカーを追加
                folium.CircleMarker(
                    location=[points[1],points[0]],
                    radius=5,
                    color=color,
                    fill=True,
                    fill_color=fill_color,
                    popup='{},{}'.format(points[0],points[1])
                ).add_to(los_map)

        #ロケットのマーカーを追加
        folium.Marker(
            location=[lat1, lon1],
            icon=folium.Icon(color='red',icon="info-sign"),
            popup='ロケット：{},{}'.format(lat1,lon1)
        ).add_to(los_map)


# #マップ作成
los_map = folium.Map(location=[lat1,lon1],zoom_start=13) 

# #発射場の標高を追加
height = height + get_kokudochiriin_elevation(lon1,lat1)

#見通しマーカー追加
draw_los_maker(lon1,lat1,height,los_radius,concentric_circle,accuracy,los_map)

# #地図を描画
los_map

見通しエリアを計算:   0%|          | 0/66 [00:00<?, ?it/s]

0.0°上の直線の見通しを計算:   0%|          | 0/11 [00:00<?, ?it/s]

5.45°上の直線の見通しを計算:   0%|          | 0/11 [00:00<?, ?it/s]